# OCR recognition with Transformers

In [ ]:
import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import requests 
from PIL import Image

model_path = "C:/Users/A/yolov5/model"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten") 
model = VisionEncoderDecoderModel.from_pretrained(model_path).to(device) 

In [ ]:
from PIL import Image
import pandas as pd
import os
images = "C:\\Users\\A\\datasets\\Thesis\\crops_aico_28\\"
df_digit = pd.DataFrame()
df_temp = pd.DataFrame(columns=['Student','Question','Subdigit','Pred'])
for img in os.listdir(images):
    image = Image.open(images+img).convert("RGB")
    pixel_values = processor(image, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values.to(device))
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    name = img.split('_')
    data = {'Student': name[0],'Question': name[1], 'Subdigit': name[2].split('.')[0],'Pred': int(generated_text)}
    df_digit = df_digit.append(data,ignore_index=True)
    

In [ ]:
df_labels = pd.read_csv('custom_label_v1.csv',dtype=str)

In [ ]:
df_merge = pd.merge(df_digit,df_labels,how='inner')

In [ ]:
df_merge = df_merge.dropna()

In [ ]:
df_merge.Pred = pd.to_numeric(df_merge.Pred, errors='coerce')
df_merge.Expected = pd.to_numeric(df_merge.Expected, errors='coerce')

In [ ]:
df_merge = df_merge[df_merge.Expected < 10].reset_index()

In [ ]:
from sklearn.metrics import confusion_matrix
y_true = df_merge.Expected
y_pred = df_merge.Pred



In [ ]:
confusion_matrix(y_true, y_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_true))